#### Domain YG에서 만든 코드를 전체 파일에 적용 시킨다. 각각의 DataFrame에 저장시킨뒤 비교
#### 딕셔너리를 이용하여 가중치를 합치는 작업을 거친다. Score = sum(tf_rof)/sum(tf) / 이기 때문에 Key값을 word로 주고 value 값을 sum(tf_rof) , sum(tf) 값으로 주면 좋겠다.
#### 명사만 추출 했으나 추후에 서술어까지 해야함 .
#### 매일매일 등락률 계산을 위해서는 등락률 계산 모듈이 필요함 Sentiment_Dic_Domain file 참고 
#### 데이터 프레임 열이름 수정 : Name -> name , Frequency -> tf 


In [1]:
import datetime
from datetime import date
import pandas as pd
import re 
import time

from konlpy.tag import Komoran
from collections import Counter

In [2]:
def getWeekday(start, end):
#start = '20200411' , end = '20200922'
    # 평일을 구하기 위한 datetime의 기본적인 변수를 생성해준다. 
    s_year = int(start[0:4]) # 2020
    s_month = int(start[5:6]) # 4
    s_day = int(start[6:8]) # 11

    e_year = int(end[0:4]) # 2020
    e_month = int(end[5:6]) # 9
    e_day = int(end[6:8]) # 22 

    day_delta = datetime.timedelta(days=1) 
    start_date = datetime.date(s_year, s_month, s_day)
    end_date = datetime.date(e_year, e_month, e_day)
    length = (end_date - start_date).days # 시작일 - 끝나는일의 일 수 
 
    # 평일 데이터만 가져오기 위해 위에서 짠 것들을 합쳐준다.
    date_weekday = [] # 평일날짜를 넣기위한 리스트 
    for i in range(length+1) : # 마지막날을 포함하기 위해 length에 + 1 을 해준다. 
        date = start_date + i*day_delta # date 는 for 문을 돌며 day_delta로 1일 씩 더해지게 된다. 
        date_w = date.weekday() # date에 저장된 날짜를 weekday로 본다 0~6의 값을 가진다. 
        if date_w < 5 : # 조건문을 걸어 0~4 월~금 만 date_weekday 리스트에 추가하도록한다. 
            date_weekday.append(str(date))
            
    return date_weekday # 평일만 포함된 date weekday 리스트가 나온다. 이를 이용해 평일 엑셀 데이터만 불러 오도록 한다. 


# 년/월/일 변수가 / 로 구분 되어있다면 data name 과 맞춰주기 위해 /를 -로 바꿔주는 부분. 
def replaceSlash(df) :
    dates = []
    df_date = df.iloc[:,:1]
    df_date = df_date.sort_values(by='date',ascending=False)
    list1 = df_date.values.tolist()
    for i in range(len(df_date)):
        tokens = list1[i][0]
        token = tokens.split('-')
        year = token[0]
        month = token[1]
        #month = month.replace('0','')
        day = token[2]
        date = year + '-' + month + '-' + day 
        dates.append(date)
    return dates

# 문장내 모든 conma를 제거해주는 함수 
def replaceConma(df, start, finish) : # start, finish 열의 인덱스를 지정해 주면된다. 
    for j in range(start, finish+1):
        for i in range(len(df.iloc[:,j])):
            df.iloc[:,j][i] = df.iloc[:,j][i].replace(",", "")
    return df
# df = replaceAllConma(df, 1, 9) 1열 부터 9열 까지의 모든 데이터의 conma를 삭제.


# user dic에 한글자가 있는 경우 이를 삭제해주는 함수 . filename 에는 userdic name 이 들어간다. 
def delDicOneword(filename) :
    with open(f'{filename}.txt', 'r' ) as f :
        line = f.readlines()
        print(line)

    # 개행문자를 포함하여 글자수가 2개 이하인 모든 글자를 삭제. 
    for i in line :
        i.strip()
        if len(i) < 3 :
            line.remove(i)

    with open(f'{filename}.txt', 'w') as f:
        for i in line:
            f.write(i)

# tf score 계산

def getTf(date) :
# date = '2020-04-13' 
    df1 = pd.read_excel(f'./data_news/YG_DATAyg {date}.xlsx')
    sentences = [] 
    list1 = list(set(df1['title'].values.tolist())) # 중복을 제거 한다.
    pattern = '[^\w\s]'
    repl = ''
    for i in list1 :
        sen = i.split(']')
        if len(sen) >= 2 : # 스플릿 했을때 ] 가 있으면 len이 2 가 나오고 없으면 1이 나옴. 그중 문장을 선택하게 조건문을 부여
            sen = sen[1]
        else :
            sen = sen[0]
        sen = re.sub(pattern," ",sen) # pattern에서 정의한 특수문자 정규표현식 을 제거 
        sen = sen.strip() # 양옆의 공백을 제거 
        sentences.append(sen)
    #sentences

    komoran_userdic=Komoran(userdic='./userdicCanSur_YG_Sentiment.txt')
    corpus = sentences
    wordCount = {}
    wordlist = []
    answer = []
    for i in corpus:
        wordlist.append(komoran_userdic.nouns(i))
    for i in wordlist:
        answer += i
    #print(answer)

    
    attention = {}
    attention = Counter(answer)
    df1 = pd.DataFrame(attention.keys())
    df1.columns =['name']
    df1['tf'] = pd.DataFrame(attention.values())
    df1 = df1[df1['tf'] >= 2] # 빈도수가 2번 이상인 단어만을 대상으로 한다. 
    df1.reset_index(inplace = True)
    df1 = df1.drop('index', axis =1)

    # 한글자인 데이터를 삭제 해준다.
    for i in range(len(df1)):
        word = df1['name'][i] # iloc로 하면 df를 drop 했을때 키를 못찾는 에러가 발생함. 
        #print(i, word)
        if len(word) < 2 : # Name 테이블 안에 글자가 한개이면 이를 삭제
            #print(i, word)
            df1.drop(index=i, inplace=True)
    return df1


def getTfrofScore(df, df1, date) :
# df는 stock data df, df1 은 news data, date는 날짜
    df = df[ df['date'] == date ] # 조건문을 걸어 변수 date 와 같은날의 행의 모든 데이터를 반환 
    rof = df['rof'].iloc[0] # 그날의 데이터중 rof값을 반환    
    df1['tf_rof'] = df1['tf']*rof # 빈도수 dataFrame에 df_rof 값을 저장
    return df1


def getSumdict(df, key_col, value_col) :
    # df = df_name , key_col = 'Name' , value_col = 'tfrof' 이렇게 key가 될 col과 value가 될 col을 정해주면 합을 구할 수 있다. 
    a_dict = {} 

    for i in range(len(df)):
        df = df.reset_index(drop=True)
        key = df[key_col][i]
        value = df[value_col][i]
        if key in a_dict:
          a_dict[key] += float(value)
        else:
          a_dict[key] = float(value)
  
    return a_dict


# 전역 변수로 설정하니까 오류가 발생함 왜그런지 모르겠음 // 내생각에는 main에 가장 처음에 변수를 주면 순서대로 실행한다고 생각했는데 그게 아닌가봄. 
# def getDf(filepath):
# filepath = './data_stock/YG_1y.csv'
#     # csv 파일로 들어가 등락률을 미리 계산해주어야함.  등락률 = 당일대비 / 전일종가
#     df = pd.read_csv(filepath)
#     df.columns = ['date', 'closing', 'diff', 'volume', 'transaction', 'market', 'high', 'low', 'capital', 'stocks', 'rof']
#     df = replaceConma(df, 1, 9)
#     # df = replaceSlash(df) # stock data date가 2020/1/3 일때 사용

1. stock data 를 불러온다 .
2. 모든 데이터의 conma를 삭제한다 .
3. stock data 의 년월일을 news data 년월일과 맞추기 위해 slash를 삭제하고 hyphen을 넣는다.  
##### 미구현 / 등락률을 계산한다. 계산후 na 값은 drop 한다.(가장 첫날에 해당함)
4. news data 를 불러온다.
##### 서술어 미구현 / 사전 정의된 Komoran 형태소 분석기를 이용해 뉴스 명사(서술어)의 Tf를 계산한다. (이 부분은 비지도 학습의 soynlp를 사용하여도 무방 하다고 생각. ) 
5. tfrof score를 구하기 위해 stock data에서 new data의 date에 맞는 등락률을 가져와 tf에 곱한다. tfrof값을 산출. 
6. 이를 데이터 프레임으로 저장. for문을 돌려 위의 과정을 반복하게 해준다. 
7. 날짜별 데이터 프레임 이름 리스트를 저장하고, 리스트를 이용하여 데이터 프레임을 concat 하여 밑으로 쭉 붙여준다.
8. 딕셔너리를 이용해서 위에서 부터 내려오면서 key 값을 체크하여 tf, tfrof 값을 추가 or 더하여 준다. 

##### 주의 사항. news file에 반드시 date가 들어가 있어야 계산이 가능하다 .

In [7]:
def main():
    
    # csv 파일로 들어가 등락률을 미리 계산해주어야함.  등락률 = 당일대비 / 전일종가
    df = pd.read_csv('./data_stock/YG_1y.csv')
    df.columns = ['date', 'closing', 'diff', 'volume', 'transaction', 'market', 'high', 'low', 'capital', 'stocks', 'rof']
    # df = replaceConma(df, 1, 9)
    # df['date'] = df['date'].astype(str)
    # df = replaceSlash(df) # stock data date가 2020/1/3 일때 사용
    
    # df1 = getTfrofScore('2020-04-11') # 하루치 Score를 구할때 
    days = getWeekday('20200411', '20200415') # 내가 기준으로 한 범위
    dfs = [] 
   
    start = time.time()
    for day in days :
        start = time.time()
        df1 = getTf(day)
        if sum(df['date'] == day) == 0:
            continue
        df_day = getTfrofScore(df, df1, day)
        print(day)
        dfs.append(df_day)
        print("time :", time.time() - start) 
        
    for i in range(1, len(dfs)):
        dfs[0] = pd.concat([dfs[0], dfs[i]])

    df_concat = dfs[0]
    tfrof_dic = getSumdict(df_concat, 'name', 'tf_rof')
    #tf_dic = getSumdict(df_concat, 'name', 'tf')
    
    
    return tfrof_dic
    #return tf_dic
    
    # 이제 여기서 나온 word 값들을 기반으로 tfrof/tf 값을 수행해주면 score 값이 나온다. 
    # keys, values 를 이용 list에 넣어서 name, tf / name , tfrof 로 변환후 acseding 시켜서 tfrof/tf 를 계산해준다.

In [8]:
if __name__ == "__main__" :
    main()

tfrof_dic

2020-04-13
time : 1.8640687465667725
2020-04-14
time : 1.363067388534546


NameError: name 'tfrof_dic' is not defined

### 위에서 main 함수에서 왜 결과 값이 안나오는지 모르겠음. 

In [11]:
# csv 파일로 들어가 등락률을 미리 계산해주어야함.  등락률 = 당일대비 / 전일종가
df = pd.read_csv('./data_stock/YG_1y.csv')
df.columns = ['date', 'closing', 'diff', 'volume', 'transaction', 'market', 'high', 'low', 'capital', 'stocks', 'rof']
# df = replaceConma(df, 1, 9)
# df['date'] = df['date'].astype(str)
# df = replaceSlash(df) # stock data date가 2020/1/3 일때 사용

# df1 = getTfrofScore('2020-04-11') # 하루치 Score를 구할때 
days = getWeekday('20200411', '20200922') # 내가 기준으로 한 범위
dfs = [] 

start = time.time()
for day in days :
    start = time.time()
    df1 = getTf(day)
    if sum(df['date'] == day) == 0:
        continue
    df_day = getTfrofScore(df, df1, day)
    print(day)
    dfs.append(df_day)
    print("time :", time.time() - start) 

for i in range(1, len(dfs)):
    dfs[0] = pd.concat([dfs[0], dfs[i]])

df_concat = dfs[0]
tfrof_dic = getSumdict(df_concat, 'name', 'tf_rof')
tf_dic = getSumdict(df_concat, 'name', 'tf')

2020-04-13
time : 1.5270659923553467
2020-04-14
time : 2.8629255294799805
2020-04-16
time : 2.3982996940612793
2020-04-17
time : 1.1920008659362793
2020-04-20
time : 1.6480445861816406
2020-04-21
time : 1.2225043773651123
2020-04-22
time : 2.515378952026367
2020-04-23
time : 1.5775227546691895
2020-04-24
time : 1.6373143196105957
2020-04-27
time : 1.692911148071289
2020-04-28
time : 3.414707899093628
2020-04-29
time : 1.581085205078125
2020-05-04
time : 1.8750665187835693
2020-05-06
time : 1.5790016651153564
2020-05-07
time : 3.9210453033447266
2020-05-08
time : 1.381105899810791
2020-05-11
time : 1.7431256771087646
2020-05-12
time : 1.520019769668579
2020-05-13
time : 2.330092191696167
2020-05-14
time : 1.884383201599121
2020-05-15
time : 1.7239983081817627
2020-05-18
time : 1.830901861190796
2020-05-19
time : 2.6856961250305176
2020-05-20
time : 1.7616510391235352
2020-05-21
time : 1.8082754611968994
2020-05-22
time : 1.93568754196167
2020-05-25
time : 2.321990966796875
2020-05-26
ti

In [12]:
tfrof_dic

{'방탄소년단': 0.35567217199999995,
 '코스': 3.4653911499999994,
 '하락': 0.46182873699999993,
 '마감': 0.487385325,
 '개인': 0.21054406299999998,
 '위너': -0.20537962499999995,
 '이승훈': -0.20242215,
 '입대': 0.17024335,
 '훈련소': -0.062283738,
 '입소': -0.249134952,
 'YG': 7.279542797,
 '상승': 1.6685974270000001,
 '트레': 0.5077582849999996,
 '지훈': -0.03355164599999999,
 '소정환': -0.00599971299999999,
 '댄스': 0.37381644300000005,
 '퍼포먼스': 0.07358629500000001,
 '공개': 0.36680564800000015,
 'GOT7': -0.217993083,
 '컴백': 0.9205086750000004,
 '쇼케이스': -0.186851214,
 '개최': -0.071399804,
 '타이틀': -0.09845079400000005,
 '무대': -0.49294168,
 '온라인': -0.119320015,
 '사인회': -0.08990543,
 '시도': -0.062283738,
 '푸드': -0.093425607,
 '전율': 0.05190311399999999,
 '화면': 0.565064594,
 '해설': 0.565064594,
 '방송': 1.2223279309999997,
 '편성': 0.6086887559999999,
 '안내': 0.565064594,
 '최선': -0.062283738,
 '김진우': 0.11834023599999999,
 '대체': -0.124567476,
 '복무': -0.124567476,
 '신곡': 1.7328155019999996,
 '최초': 0.6788495239999999,
 '세븐': -0.30044336

In [14]:
tf_dic

{'방탄소년단': 72.0,
 '코스': 129.0,
 '하락': 38.0,
 '마감': 12.0,
 '개인': 41.0,
 '위너': 61.0,
 '이승훈': 15.0,
 '입대': 8.0,
 '훈련소': 2.0,
 '입소': 8.0,
 'YG': 635.0,
 '상승': 105.0,
 '트레': 405.0,
 '지훈': 12.0,
 '소정환': 14.0,
 '댄스': 27.0,
 '퍼포먼스': 11.0,
 '공개': 367.0,
 'GOT7': 7.0,
 '컴백': 159.0,
 '쇼케이스': 6.0,
 '개최': 11.0,
 '타이틀': 24.0,
 '무대': 23.0,
 '온라인': 9.0,
 '사인회': 6.0,
 '시도': 2.0,
 '푸드': 3.0,
 '전율': 4.0,
 '화면': 26.0,
 '해설': 26.0,
 '방송': 56.0,
 '편성': 28.0,
 '안내': 26.0,
 '최선': 2.0,
 '김진우': 4.0,
 '대체': 4.0,
 '복무': 4.0,
 '신곡': 124.0,
 '최초': 56.0,
 '세븐': 13.0,
 '개념': 4.0,
 '주가': 66.0,
 '최근': 7.0,
 '종목': 45.0,
 '박진영': 12.0,
 '작사': 8.0,
 '작곡': 8.0,
 '편곡': 4.0,
 '참여': 22.0,
 '논산': 2.0,
 '사회복무요원': 5.0,
 '급등': 49.0,
 '사랑': 29.0,
 '엔터': 151.0,
 '논산훈련소': 2.0,
 '콘텐츠': 12.0,
 '출신': 25.0,
 'SM': 63.0,
 '카라': 4.0,
 '배우': 10.0,
 '한승연': 7.0,
 '전속': 15.0,
 '계약': 33.0,
 '체결': 12.0,
 '이수혁': 12.0,
 '주우': 2.0,
 '한솥밥': 11.0,
 '지드래곤': 66.0,
 '음료': 2.0,
 '촬영': 17.0,
 '기준': 10.0,
 '문화': 4.0,
 '네이버': 24.0,
 '글로벌': 66.0,
 '시장': 14.0,

In [36]:
tf_dic.keys() == tfrof_dic.keys()

True

In [38]:

df2 = pd.DataFrame(tf_dic.keys())
df2.columns = ['keys']

df2['tfrof'] = tfrof_dic.values()
df2['tf'] = tf_dic.values()

df2['score'] = df2['tfrof']/df2['tf']

df2

,keys,tfrof,tf,score
0,방탄소년단,0.355672,72.0,0.004940
1,코스,3.465391,129.0,0.026863
2,하락,0.461829,38.0,0.012153
3,마감,0.487385,12.0,0.040615
4,개인,0.210544,41.0,0.005135
...,...,...,...,...
1302,타투,-0.082759,2.0,-0.041379
1303,예사,-0.082759,2.0,-0.041379
1304,빌딩,-0.165517,4.0,-0.041379
1305,개선,-0.021583,3.0,-0.007194


In [54]:
# 구한 값을 저장해 주어야함. lexicon 형태로 .
data = df2.loc[:, ['keys', 'score']]
data

,keys,score
0,방탄소년단,0.004940
1,코스,0.026863
2,하락,0.012153
3,마감,0.040615
4,개인,0.005135
...,...,...
1302,타투,-0.041379
1303,예사,-0.041379
1304,빌딩,-0.041379
1305,개선,-0.007194


In [55]:
data.to_csv('vader_lexicon_kr_yg.txt', index=False, header=None, sep="\t")

True